In [ ]:
# 01 - Análisis y ETL con Pandas

#Este notebook realiza la exploración, limpieza y el proceso ETL del dataset de videojuegos usando Pandas y carga el resultado a una base de datos SQLite (`warehouse/warehouse_pandas.db`).

#Asegúrate de ejecutar este notebook desde la raíz del proyecto para que las rutas relativas funcionen correctamente.


In [1]:
import os
import re
import numpy as np
import pandas as pd
import sqlalchemy

# -------------------------------
# Mensajes de información
# -------------------------------

print("Librerías cargadas correctamente:")
print(" - os:", os)
print(" - re (expresiones regulares):", re)
print(" - numpy:", np.__version__)
print(" - pandas:", pd.__version__)
print(" - sqlalchemy:", sqlalchemy.__version__)
print("\nPreparado para cargar y procesar el dataset de videojuegos.\n")

Librerías cargadas correctamente:
 - os: <module 'os' from '/usr/local/lib/python3.9/os.py'>
 - re (expresiones regulares): <module 're' from '/usr/local/lib/python3.9/re.py'>
 - numpy: 2.0.2
 - pandas: 2.3.3
 - sqlalchemy: 2.0.45

Preparado para cargar y procesar el dataset de videojuegos.



In [2]:
# -------------------------------
# 1️ Carga del dataset
# -------------------------------

# Se carga el CSV desde la carpeta /data
raw_df = pd.read_csv('../data/videogames.csv')

# Mensaje informativo y revisión inicial
print("Dataset cargado correctamente desde '../data/videogames.csv'")
print(f"Dimensiones del dataset: {raw_df.shape[0]} filas x {raw_df.shape[1]} columnas")
print("Primeras 5 filas del dataset:")
print(raw_df.head())

Dataset cargado correctamente desde '../data/videogames.csv'
Dimensiones del dataset: 10000 filas x 21 columnas
Primeras 5 filas del dataset:
                  name    genre   cost platform         popularity pegi  year  \
0  Super Mario Odyssey   Action  74.45   Mobile                 56   7+  2011   
1           God of War     RPG       0   Mobile                  ?   7+  2023   
2      Persona 5 Royal  Shooter   Free       PS                 64   12  2020   
3             NBA 2K24   Puzzle    NaN   Mobile  972.7113240416031   RP  2017   
4            Overwatch        ?   33.4       PC  612.6268621737502  18+  2015   

  developer     publisher   region  ...         engine        award  \
0    Capcom   Square Enix        ?  ...      CryEngine  Indie Award   
1  Rockstar      Nintendo  Global   ...          Unity            ?   
2  nintendo   Square Enix      NaN  ...  Custom Engine         GotY   
3      Sony   Square Enix  Global   ...  Custom Engine         NONE   
4  Nintendo  Ban

In [3]:
# Inspección inicial de los datos
print("===== Información general del dataset =====")
raw_df.info() # Muestra tipos de datos, nulos y número de filas/columnas

print("\n===== Estadísticas descriptivas =====")
print(raw_df.describe(include="all").T) # Muestra estadísticas descriptivas de todas las columnas (numéricas y categóricas)

===== Información general del dataset =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  9714 non-null   object
 1   genre                 10000 non-null  object
 2   cost                  8350 non-null   object
 3   platform              10000 non-null  object
 4   popularity            7467 non-null   object
 5   pegi                  10000 non-null  object
 6   year                  9768 non-null   object
 7   developer             10000 non-null  object
 8   publisher             10000 non-null  object
 9   region                8602 non-null   object
 10  mode                  10000 non-null  object
 11  engine                10000 non-null  object
 12  award                 8578 non-null   object
 13  dlc_support           10000 non-null  object
 14  language              10000 non-null  objec

In [5]:
# -------------------------------
# 2️ Limpieza de datos
# -------------------------------

print("===== 1️ Eliminación de duplicados =====")
clean_df = raw_df.drop_duplicates().copy()
print(f"Filas después de eliminar duplicados: {clean_df.shape[0]}")

# -------------------------------
# Identificación de tipos de columnas
# -------------------------------
numeric_cols = clean_df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = clean_df.select_dtypes(include=["object"]).columns.tolist()

# --------------------------------------------------
# Limpieza de columnas numéricas problemáticas
# --------------------------------------------------
def parse_numeric(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, str):
        val = val.strip().replace(",", "").replace("$", "").upper()
        if val in ["UNKNOWN", "?"]:
            return np.nan
        if val.endswith("M"):
            try: return float(val[:-1]) * 1e6
            except: return np.nan
        if val.endswith("B"):
            try: return float(val[:-1]) * 1e9
            except: return np.nan
    try:
        return float(val)
    except:
        return np.nan

print("\n===== 2 Limpieza de columnas de ventas/ingresos =====")
for col in ["copies_sold_millions", "revenue_millions_usd"]:
    if col in clean_df.columns:
        # Aplicamos la función de limpieza
        clean_df[col] = clean_df[col].apply(parse_numeric)
        # Convertimos a tipo float explícitamente
        clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')
        # Rellenamos los NaN con la media
        clean_df[col] = clean_df[col].fillna(clean_df[col].mean())
        print(f"Columna '{col}' limpiada. Valores nulos restantes: {clean_df[col].isna().sum()}")

# --------------------------------------------------
# Imputación de columnas categóricas
# --------------------------------------------------
cat_cols = clean_df.select_dtypes(include=["object"]).columns.tolist()
print("\n===== 3 Imputación de columnas categóricas =====")
for col in cat_cols:
    clean_df[col] = clean_df[col].fillna("Unknown")
    print(f"Columna '{col}' rellenada con 'Unknown'. Valores nulos restantes: {clean_df[col].isna().sum()}")

# Resumen final
print("\n===== Resumen final de limpieza =====")
print(clean_df.isna().sum())
print(f"Dimensiones del dataset limpio: {clean_df.shape[0]} filas x {clean_df.shape[1]} columnas")

===== 1️ Eliminación de duplicados =====
Filas después de eliminar duplicados: 10000

===== 2 Limpieza de columnas de ventas/ingresos =====
Columna 'copies_sold_millions' limpiada. Valores nulos restantes: 0
Columna 'revenue_millions_usd' limpiada. Valores nulos restantes: 0

===== 3 Imputación de columnas categóricas =====
Columna 'name' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'genre' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'cost' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'platform' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'popularity' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'pegi' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'year' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'developer' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'publisher' rellenada con 'Unknown'. Valores nulos restantes: 0
Columna 'region' rellenada con 'Unknown'. Valores 

In [6]:
# -------------------------------
# 3️ Normalización de columnas
# -------------------------------

print("===== Normalización de nombres de columnas =====")
print("Columnas antes de normalizar:")
print(list(raw_df.columns))

# Convertimos nombres a snake_case para facilidad de uso
clean_df.columns = (
    clean_df.columns
    .str.strip()             # Elimina espacios al inicio y final
    .str.lower()             # Convierte a minúsculas
    .str.replace(" ", "_", regex=False)  # Reemplaza espacios por '_'
    .str.replace("-", "_", regex=False)  # Reemplaza '-' por '_'
)

print("\nColumnas después de normalizar a snake_case:")
print(list(clean_df.columns))

===== Normalización de nombres de columnas =====
Columnas antes de normalizar:
['name', 'genre', 'cost', 'platform', 'popularity', 'pegi', 'year', 'developer', 'publisher', 'region', 'mode', 'engine', 'award', 'dlc_support', 'language', 'metascore', 'user_score', 'reviews', 'rating_source', 'copies_sold_millions', 'revenue_millions_usd']

Columnas después de normalizar a snake_case:
['name', 'genre', 'cost', 'platform', 'popularity', 'pegi', 'year', 'developer', 'publisher', 'region', 'mode', 'engine', 'award', 'dlc_support', 'language', 'metascore', 'user_score', 'reviews', 'rating_source', 'copies_sold_millions', 'revenue_millions_usd']


In [7]:
# -------------------------------
# 4️ Construcción de tablas dimensionales
# -------------------------------

print("===== Validación de columnas requeridas =====")
required_cols = ["name", "genre", "platform", "developer", "publisher", "year"]
for c in required_cols:
    if c not in clean_df.columns:
        raise ValueError(f"Columna requerida no encontrada en el CSV: {c}")
print("Todas las columnas requeridas están presentes.\n")

# -------------------------------
# Dimensión: Videojuegos
# -------------------------------
dim_game = clean_df[["name", "genre"]].drop_duplicates().reset_index(drop=True)
dim_game["id_game"] = np.arange(1, len(dim_game) + 1)
print("Dimensión 'dim_game' creada:")
print(f"Filas: {dim_game.shape[0]}")
print(dim_game.head())

# -------------------------------
# Dimensión: Plataformas
# -------------------------------
dim_platform = clean_df[["platform"]].drop_duplicates().reset_index(drop=True)
dim_platform["id_platform"] = np.arange(1, len(dim_platform) + 1)
print("\nDimensión 'dim_platform' creada:")
print(f"Filas: {dim_platform.shape[0]}")
print(dim_platform.head())

# -------------------------------
# Dimensión: Desarrolladores
# -------------------------------
dim_developer = clean_df[["developer"]].drop_duplicates().reset_index(drop=True)
dim_developer["id_developer"] = np.arange(1, len(dim_developer) + 1)
print("\nDimensión 'dim_developer' creada:")
print(f"Filas: {dim_developer.shape[0]}")
print(dim_developer.head())

# -------------------------------
# Dimensión: Publisher
# -------------------------------
dim_publisher = clean_df[["publisher"]].drop_duplicates().reset_index(drop=True)
dim_publisher["id_publisher"] = np.arange(1, len(dim_publisher) + 1)
print("\nDimensión 'dim_publisher' creada:")
print(f"Filas: {dim_publisher.shape[0]}")
print(dim_publisher.head())

# -------------------------------
# Dimensión: Año
# -------------------------------
dim_year = clean_df[["year"]].drop_duplicates().reset_index(drop=True)
dim_year["id_year"] = np.arange(1, len(dim_year) + 1)
print("\nDimensión 'dim_year' creada:")
print(f"Filas: {dim_year.shape[0]}")
print(dim_year.head())

===== Validación de columnas requeridas =====
Todas las columnas requeridas están presentes.

Dimensión 'dim_game' creada:
Filas: 490
                  name    genre  id_game
0  Super Mario Odyssey   Action        1
1           God of War     RPG         2
2      Persona 5 Royal  Shooter        3
3             NBA 2K24   Puzzle        4
4            Overwatch        ?        5

Dimensión 'dim_platform' creada:
Filas: 9
          platform  id_platform
0           Mobile            1
1               PS            2
2               PC            3
3  Nintendo Switch            4
4          Switch             5

Dimensión 'dim_developer' creada:
Filas: 14
  developer  id_developer
0    Capcom             1
1  Rockstar             2
2  nintendo             3
3      Sony             4
4  Nintendo             5

Dimensión 'dim_publisher' creada:
Filas: 13
      publisher  id_publisher
0   Square Enix             1
1      Nintendo             2
2  Bandai Namco             3
3          Sega    

In [8]:
# -------------------------------
# 5️ Construcción de tabla de hechos
# -------------------------------

print("===== Unión de dimensiones para crear la tabla de hechos =====")
# Se unen todas las dimensiones al dataset original para generar la tabla de hechos
fact = clean_df.merge(dim_game, on=["name", "genre"], how="left")
fact = fact.merge(dim_platform, on=["platform"], how="left")
fact = fact.merge(dim_developer, on=["developer"], how="left")
fact = fact.merge(dim_publisher, on=["publisher"], how="left")
fact = fact.merge(dim_year, on=["year"], how="left")
print("Unión completada. Dimensiones del dataset de hechos:", fact.shape)

# -------------------------------
# Selección de columnas métricas
# -------------------------------
# Verificamos qué columnas de ventas/ingresos existen en el dataset
value_cols = [c for c in ["copies_sold_millions", "revenue_millions_usd"] if c in fact.columns]
if not value_cols:
    raise ValueError("No se encontraron columnas de ventas/ingresos en el CSV.")
print("Columnas métricas seleccionadas para la tabla de hechos:", value_cols)

# -------------------------------
# Tabla de hechos final
# -------------------------------
fact_sales = fact[["id_game", "id_platform", "id_developer", "id_publisher", "id_year"] + value_cols].copy()
print("\nTabla de hechos 'fact_sales' creada:")
print(f"Dimensiones: {fact_sales.shape[0]} filas x {fact_sales.shape[1]} columnas")
print("Primeros registros de la tabla de hechos:")
print(fact_sales.head())


===== Unión de dimensiones para crear la tabla de hechos =====
Unión completada. Dimensiones del dataset de hechos: (10000, 26)
Columnas métricas seleccionadas para la tabla de hechos: ['copies_sold_millions', 'revenue_millions_usd']

Tabla de hechos 'fact_sales' creada:
Dimensiones: 10000 filas x 7 columnas
Primeros registros de la tabla de hechos:
   id_game  id_platform  id_developer  id_publisher  id_year  \
0        1            1             1             1        1   
1        2            1             2             2        2   
2        3            2             3             1        3   
3        4            1             4             1        4   
4        5            3             5             3        5   

   copies_sold_millions  revenue_millions_usd  
0          4.193000e+01          5.073936e+08  
1          1.500000e+06          5.073936e+08  
2          2.508000e+01          8.890000e+02  
3          1.030856e+07          5.000000e+08  
4          1.030856e+07

In [9]:
# -------------------------------
# 6️ Carga en SQLite
# -------------------------------

print("===== 1 Creación de carpeta y configuración de la base de datos =====")
# Creamos la carpeta warehouse si no existe
os.makedirs("../warehouse", exist_ok=True)
DB_PATH = "../warehouse/warehouse_pandas.db"
DB_URL = f"sqlite:///{DB_PATH}"
print(f"Base de datos SQLite se guardará en: {DB_PATH}")

# Creamos el engine para SQLite
engine = sqlalchemy.create_engine(DB_URL)

# -------------------------------
# Guardado de tablas dimensionales y tabla de hechos
# -------------------------------
print("\n===== 2️ Guardado de tablas en SQLite =====")
with engine.begin() as conn:
    dim_game.to_sql("dim_game", conn, if_exists="replace", index=False)
    print(" - dim_game cargada")
    dim_platform.to_sql("dim_platform", conn, if_exists="replace", index=False)
    print(" - dim_platform cargada")
    dim_developer.to_sql("dim_developer", conn, if_exists="replace", index=False)
    print(" - dim_developer cargada")
    dim_publisher.to_sql("dim_publisher", conn, if_exists="replace", index=False)
    print(" - dim_publisher cargada")
    dim_year.to_sql("dim_year", conn, if_exists="replace", index=False)
    print(" - dim_year cargada")
    fact_sales.to_sql("fact_sales", conn, if_exists="replace", index=False)
    print(" - fact_sales cargada")

print("\n Todas las tablas fueron cargadas correctamente en SQLite.")

===== 1 Creación de carpeta y configuración de la base de datos =====
Base de datos SQLite se guardará en: ../warehouse/warehouse_pandas.db

===== 2️ Guardado de tablas en SQLite =====
 - dim_game cargada
 - dim_platform cargada
 - dim_developer cargada
 - dim_publisher cargada
 - dim_year cargada
 - fact_sales cargada

 Todas las tablas fueron cargadas correctamente en SQLite.


In [10]:
# -------------------------------
# 7️ Consulta analítica
# -------------------------------

print("===== Consulta: Top 10 géneros por ventas =====")

# Consulta SQL para obtener los géneros con mayores ventas en millones de copias
query = """
SELECT g.genre, SUM(f.copies_sold_millions) AS total_sales
FROM fact_sales f
JOIN dim_game g ON f.id_game = g.id_game
GROUP BY g.genre
ORDER BY total_sales DESC
LIMIT 10;
"""
print("Consulta SQL ejecutada:\n", query)

# Ejecutamos la consulta y la cargamos en un DataFrame de Pandas
with engine.connect() as conn:
    top_genres = pd.read_sql(query, conn)

# Mostramos resultados
print("\n===== Resultado: Top 10 géneros por ventas =====")
print(top_genres)


===== Consulta: Top 10 géneros por ventas =====
Consulta SQL ejecutada:
 
SELECT g.genre, SUM(f.copies_sold_millions) AS total_sales
FROM fact_sales f
JOIN dim_game g ON f.id_game = g.id_game
GROUP BY g.genre
ORDER BY total_sales DESC
LIMIT 10;


===== Resultado: Top 10 géneros por ventas =====
       genre   total_sales
0          ?  8.010624e+09
1  Adventure  7.897487e+09
2       RPG   7.680849e+09
3     Racing  7.627009e+09
4    Unknown  7.614050e+09
5     Puzzle  7.557338e+09
6        RPG  7.496732e+09
7     Action  7.435295e+09
8      Indie  7.223987e+09
9     Sports  7.216199e+09
